In [3]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [4]:
with open('interview.json') as file:
    data = json.load(file)

In [5]:
from sklearn.preprocessing import LabelEncoder


In [7]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [8]:

enc = LabelEncoder()
enc.fit(training_labels)
training_labels = enc.transform(training_labels)

In [9]:
vocab_size = 10000
embedding_dim = 16
max_len = 20
trunc_type = 'post'
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, truncating=trunc_type, maxlen=max_len)

In [10]:
classes = len(labels)

In [11]:
model = tf.keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(classes, activation='softmax'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
Total params: 160,714
Trainable params: 160,714
Non-trainable params: 0
_________________________________________________________________


In [13]:
training_labels_final = np.array(training_labels)

In [14]:
EPOCHS = 500
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(padded, training_labels_final, epochs=EPOCHS)

Epoch 299/500
2/2 [==============================] - 0s 4ms/step - loss: 0.6848 - accuracy: 0.9500
Epoch 300/500
2/2 [==============================] - 0s 4ms/step - loss: 0.6830 - accuracy: 0.9250
Epoch 301/500
2/2 [==============================] - 0s 5ms/step - loss: 0.6789 - accuracy: 0.9250
Epoch 302/500
2/2 [==============================] - 0s 4ms/step - loss: 0.6744 - accuracy: 0.9250
Epoch 303/500
2/2 [==============================] - 0s 4ms/step - loss: 0.6698 - accuracy: 0.9250
Epoch 304/500
2/2 [==============================] - 0s 3ms/step - loss: 0.6654 - accuracy: 0.9250
Epoch 305/500
2/2 [==============================] - 0s 4ms/step - loss: 0.6620 - accuracy: 0.9250
Epoch 306/500
2/2 [==============================] - 0s 4ms/step - loss: 0.6595 - accuracy: 0.9250
Epoch 307/500
2/2 [==============================] - 0s 4ms/step - loss: 0.6536 - accuracy: 0.9250
Epoch 308/500
2/2 [==============================] - 0s 3ms/step - loss: 0.6490 - accuracy: 0.9250
Epoch 309/

In [15]:
def check():
    print('start talking with bot, Enter quit to exit')
    while True:
        string = input('Enter: ')
        if string == 'quit': break
        result = model.predict(pad_sequences(tokenizer.texts_to_sequences([string]),
                                             truncating=trunc_type, maxlen=max_len))
        category = enc.inverse_transform([np.argmax(result)]) # labels[np.argmax(result)]
        for i in data['intents']:
            if i['tag']==category:
                print(np.random.choice(i['responses']))

In [16]:
check()

start talking with bot, Enter quit to exit
Hi there, how can I help?
Hi there, how can I help?
22 years young!
22 years young!
Kunal Duran is all about providing value and becoming more valuable every single Day. In love with Cricket since childhood and started programming and Analysis to connect the dots between Data and Improvement in the Game. Now I IDEA(Inhale Data Exhale Analysis)
I'm Kunal!
Hi there, how can I help?
Hello!
Sad to see you go :(
Good to see you again!
Sad to see you go :(
Sad to see you go :(
Good to see you again!
Hello!
Hello!
Hi there, how can I help?


In [18]:
tf.keras.models.save_model(model, "chatbot_model")

INFO:tensorflow:Assets written to: chatbot_model\assets
